In [1]:
# import libraries
import numpy as np
import pandas as pd
import h5py
import json
from datetime import datetime
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib import ticker, cm, colors
from matplotlib.lines import Line2D
from matplotlib.patches import Patch
from colormap import rgb2hex
import seaborn as sns
from itertools import compress
import os
from scipy.interpolate import interp1d
from datetime import datetime
from calfews_src import *
from calfews_src.util import *
# from calfews_src.visualizer import Visualizer

mpl.rcParams.update({'font.size': 12})

In [2]:
scenarios = ['median_none','median_friantHistorical']#, 'median_friant', 'median_none', 'wet_friant', 'wet_none']#, 'median_all', 'median_none']#, 'FKC_rehab_ownership_LWT']

shares = {}
for s in scenarios:
    shares[s] = json.load(open('FKC_experiment/FKC_experiment_capow_50yr_' + s + '/FKC_scenario.json'))['ownership_shares']
    shares_CFWB = json.load(open('FKC_experiment/FKC_experiment_capow_50yr_' + s + '/CFWB_scenario.json'))['ownership']
    for k in shares[s]:
        try:
            shares[s][k] = max(shares[s][k], shares_CFWB[k])
        except:
            pass
        
district_lookup = {
    'BDM': 'berrenda',
    'BLR': 'belridge',
    'BVA': 'buenavista',
    'CWO': 'cawelo',
    'HML': 'henrymiller',
    'ID4': 'ID4',
    'KND': 'kerndelta',
    'LHL': 'losthills',
    'RRB': 'rosedale',
    'SMI': 'semitropic',
    'THC': 'tehachapi',
    'TJC': 'tejon',
    'WKN': 'westkern',
    'WRM': 'wheeler',
    'KCWA': 'kcwa',
    'COB': 'bakersfield',
    'NKN': 'northkern',
    'ARV': 'arvin',
    'PIX': 'pixley',
    'DLE': 'delano',
    'EXE': 'exeter',
    'KRT': 'kerntulare',
    'LND': 'lindmore',
    'LDS': 'lindsay',
    'LWT': 'lowertule',
    'PRT': 'porterville',
    'SAU': 'saucelito',
    'SFW': 'shaffer',
    'SSJ': 'sosanjoaquin',
    'TPD': 'teapot',
    'TBA': 'terra',
    'TUL': 'tulare',
    'COF': 'fresno',
    'FRS': 'fresnoid',
    'SOC': 'socal',
    'SOB': 'southbay',
    'CCA': 'centralcoast',
    'DLR': 'dudleyridge',
    'TLB': 'tularelake',
    'KWD': 'kaweahdelta',
    'WSL': 'westlands',
    'SNL': 'sanluiswater',
    'PNC': 'panoche',
    'DLP': 'delpuerto',
    'CWC': 'chowchilla',
    'MAD': 'madera',
    'OTL': 'othertule',
    'OFK': 'otherfriant',
    'OCD': 'othercvp',
    'OEX': 'otherexchange',
    'OXV': 'othercrossvalley',
    'OSW': 'otherswp',
    'CNS': 'consolidated',
    'ALT': 'alta',
    'KRWA': 'krwa'
}
district_lookup_rev = {v:k for k,v in district_lookup.items()}

In [3]:
results_default = pd.read_csv('figures/results_default.csv')

In [4]:
district_res = {}
for i, s in enumerate(scenarios):
    district_res[s] = {}
    dat = get_results_sensitivity_number_outside_model('FKC_experiment/FKC_experiment_capow_50yr_' + s + '/results.hdf5', '')
    keys = list(dat.keys())
    index = dat.index
    year = index.year
    month = index.month
    dom = index.day
    doy = index.dayofyear
    dowy = (doy + (365-274)) % 365
    wy = np.array([year[i] if month[i] < 10 else year[i] + 1 for i in range(len(year))])
    for d, v in district_lookup.items():
        b = [k for k in keys if (((d in k.split('_')) or (v in k.split('_'))) and (('delivery' in k.split('_')) or 
                                                                                   ('flood' in k.split('_')) or 
                                                                                   ('recharged' in k.split('_')) or 
                                                                                   ('exchanged' in k.split('_')) or 
                                                                                   ('inleiu' in k.split('_')) or 
                                                                                   ('leiupumping' in k.split('_')) or 
                                                                                   ('banked' in k.split('_'))))]
        df = dat.loc[:, b]
        ## undo summation over years
        for y in range(df.index.year.min() + 2, df.index.year.max() + 1):
            maxprevious = df.loc[wy < y, :].iloc[-1, :]
            df.loc[wy == y, :] += maxprevious
        district_res[s][d] = df.diff()
    del dat, df
    

In [5]:
def get_district_color(dkey):
    color = results_default.color.loc[results_default.district == dkey].values[0]
    color = color.split('(')[1]
    color = color.split(')')[0]
    color = color.split(', ')
    color = [int(255 * float(c)) for c in color]
    color = rgb2hex(color[0], color[1], color[2])
    return color

In [6]:
### first get total flows b4 improvement, separated by contract
dlist = []
clist = []
alist = []
slist = []
colorlist = []
for dkey in district_res[scenarios[1]]:
    d = district_lookup[dkey]
    d_anon = results_default.district_label.loc[results_default.district == dkey].values[0]
    color = get_district_color(dkey)
    try:
        dres = district_res[scenarios[0]][dkey]
        dres = dres.sum()
    except:
        pass
    # divide by 50 to get annual changes
    dres /= 50
    for clabel, cs in zip(['Friant', 'Local', 'TableA'],
                          [['friant1', 'friant2'], ['kern', 'tule', 'kaweah', 'kings'], ['tableA']]):
        amt = 0.0
        for c in cs:
            dc = d + '_' + c + '_'
            for mode in ['delivery', 'flood', 'flood_irrigation']:
                try:
                    amt += dres[dc + mode]
                except:
                    pass
        if amt > 0.0:
            dlist.append('+' + d_anon)
            clist.append('+' + clabel)
            alist.append(amt)
            slist.append(1)
            colorlist.append(color)

        elif amt < 0.0:
            dlist.append('-' + d_anon)
            clist.append('-' + clabel)
            alist.append(abs(amt))
            slist.append(-1)
            colorlist.append(color)
            
    ## get "other" = cvpdelta, exchange contractors, cvc, & adjustments for GW/SW banking exchanges
    amt = 0.0
    for c in ['cvpdelta', 'exchange', 'cvc']:
        dc = d + '_' + c + '_'
        for mode in ['delivery', 'flood', 'flood_irrigation']:
            try:
                amt += dres[dc + mode]
            except:
                pass
#     for cs, sign in zip([['exchanged_SW','exchanged_GW']], [[1,-1]]):
#         for i,c in enumerate(cs):
#             dc = d + '_' + c
#             try:
#                 amt += dres[dc] * sign[i]
#             except:
#                 pass
    if amt > 0.0:
        dlist.append('+' + d_anon)
        clist.append('+' + 'Other')
        alist.append(amt)
        slist.append(1)
        colorlist.append(color)

    elif amt < 0.0:
        dlist.append('-' + d_anon)
        clist.append('-' + 'Other')
        alist.append(abs(amt))
        slist.append(-1)
        colorlist.append(color)
                
df = pd.DataFrame({'Source': clist, 'District': dlist, 'Magnitude': alist, 'Sign': slist, 'Color': colorlist})
print(df)
df.to_csv('figures/alluvial_contract_b4.csv', index=False)

      Source District   Magnitude  Sign    Color
0    +Friant    +O-10    0.045822     1  #A13403
1     +Local    +O-10    1.535529     1  #A13403
2    +TableA    +O-10   28.092245     1  #A13403
3    +Friant    +O-26    0.018111     1  #E45709
4     +Local    +O-26    0.598221     1  #E45709
..       ...      ...         ...   ...      ...
114   +Local    +O-23    0.246625     1  #DD4E04
115  +TableA    +O-17   13.961883     1  #C44001
116   +Local     +O-7  226.881294     1  #962F03
117   +Local     +O-8  128.319737     1  #993103
118   +Local     +O-3  807.965988     1  #862903

[119 rows x 5 columns]


In [7]:
### now get gains/losses by contract
dlist = []
clist = []
alist = []
slist = []
colorlist = []
for dkey in district_res[scenarios[1]]:
    d = district_lookup[dkey]
    d_anon = results_default.district_label.loc[results_default.district == dkey].values[0]
    color = get_district_color(dkey)
    try:
        dres = district_res[scenarios[1]][dkey] - district_res[scenarios[0]][dkey]
        dres = dres.sum()
    except:
        try:
            dres = district_res[scenarios[1]][dkey]
            dres = dres.sum()
        except:
            try:
                dres = -district_res[scenarios[0]][dkey]
                dres = dres.sum()
            except:
                pass
    # divide by 50 to get annual changes
    dres /= 50
    for clabel, cs in zip(['Friant', 'Local', 'TableA'],
                          [['friant1', 'friant2'], ['kern', 'tule', 'kaweah', 'kings'], ['tableA']]):
        amt = 0.0
        for c in cs:
            dc = d + '_' + c + '_'
            for mode in ['delivery', 'flood', 'flood_irrigation']:
                try:
                    amt += dres[dc + mode]
                except:
                    pass
        if amt > 0.0:
            dlist.append('+' + d_anon)
            clist.append('+' + clabel)
            alist.append(amt)
            slist.append(1)
            colorlist.append(color)

        elif amt < 0.0:
            dlist.append('-' + d_anon)
            clist.append('-' + clabel)
            alist.append(abs(amt))
            slist.append(-1)
            colorlist.append(color)
            
    ## get "other" = cvpdelta, exchange contractors, cvc, & adjustments for GW/SW banking exchanges
    amt = 0.0
    for c in ['cvpdelta', 'exchange', 'cvc']:
        dc = d + '_' + c + '_'
        for mode in ['delivery', 'flood', 'flood_irrigation']:
            try:
                amt += dres[dc + mode]
            except:
                pass
#     for cs, sign in zip([['exchanged_SW','exchanged_GW']], [[1,-1]]):
#         for i,c in enumerate(cs):
#             dc = d + '_' + c
#             try:
#                 amt += dres[dc] * sign[i]
#             except:
#                 pass
    if amt > 0.0:
        dlist.append('+' + d_anon)
        clist.append('+' + 'Other')
        alist.append(amt)
        slist.append(1)
        colorlist.append(color)

    elif amt < 0.0:
        dlist.append('-' + d_anon)
        clist.append('-' + 'Other')
        alist.append(abs(amt))
        slist.append(-1)
        colorlist.append(color)
                
df = pd.DataFrame({'Source': clist, 'District': dlist, 'Magnitude': alist, 'Sign': slist, 'Color': colorlist})
print(df)
df.to_csv('figures/alluvial_contract.csv', index=False)

      Source District  Magnitude  Sign    Color
0    -Friant    -O-10   0.000514    -1  #A13403
1     -Local    -O-10   0.025197    -1  #A13403
2    +TableA    +O-10   0.034313     1  #A13403
3    +Friant    +O-26   0.000044     1  #E45709
4     -Local    -O-26   0.007599    -1  #E45709
..       ...      ...        ...   ...      ...
114   +Local    +O-23   0.001054     1  #DD4E04
115  -TableA    -O-17   0.004147    -1  #C44001
116   +Local     +O-7   0.053090     1  #962F03
117   +Local     +O-8   0.030592     1  #993103
118   +Local     +O-3   0.312290     1  #862903

[119 rows x 5 columns]


In [8]:
(df.Magnitude * df.Sign).sum()

35.87455189420754

In [9]:
### total flows by month, b4 project
dlist = []
alist = []
slist = []
mlist = []
colorlist = []
for m in range(1,13):
    for dkey in district_res[scenarios[1]]:
        d = district_lookup[dkey]
        d_anon = results_default.district_label.loc[results_default.district == dkey].values[0]
        color = get_district_color(dkey)
        try:
            dres = district_res[scenarios[0]][dkey]
            dres = dres.loc[dres.index.month == m].sum()
        except:
            pass
        # divide by 50 to get annual changes
        dres /= 50
        amt = 0.0
        for c in ['friant1', 'friant2', 'cvpdelta', 'exchange', 'cvc', 'kern', 'tule', 'kaweah', 'kings', 'tableA']:
            dc = d + '_' + c + '_'
            for mode in ['delivery', 'flood', 'flood_irrigation']:
                try:
                    amt += dres[dc + mode]
                except:
                    pass
        for cs, sign in zip([['exchanged_SW','exchanged_GW']], [[1,-1]]):
            for i,c in enumerate(cs):
                dc = d + '_' + c
                try:
                    amt += dres[dc] * sign[i]
                except:
                    pass
        if amt > 0.0:
            dlist.append('+' + d_anon)
            alist.append(amt)
            slist.append(1)
            mlist.append('+' + str(m))
            colorlist.append(color)

        elif amt < 0.0:
            dlist.append('-' + d_anon)
            alist.append(abs(amt))
            slist.append(-1)
            mlist.append('-' + str(m))
            colorlist.append(color)
                
                
            
                
                
df = pd.DataFrame({'Month': mlist, 'District': dlist, 'Magnitude': alist, 'Sign': slist, 'Color': colorlist})
print(df)
df.to_csv('figures/alluvial_month_b4.csv', index=False)

    Month District  Magnitude  Sign    Color
0      +1    +O-10   2.244580     1  #A13403
1      +1    +O-26   1.014048     1  #E45709
2      +1     +O-6   8.505698     1  #912E03
3      +1    +O-29   2.334769     1  #EC620F
4      +1    +O-24   2.029901     1  #DF5106
..    ...      ...        ...   ...      ...
643   +12    +O-23   0.150921     1  #DD4E04
644   +12    +O-17   1.320401     1  #C44001
645   +12     +O-7   7.971902     1  #962F03
646   +12     +O-8   3.972135     1  #993103
647   +12     +O-3   5.240026     1  #862903

[648 rows x 5 columns]


In [10]:
### now gain/loss by month
dlist = []
alist = []
slist = []
mlist = []
colorlist = []
for m in range(1,13):
    for dkey in district_res[scenarios[1]]:
        d = district_lookup[dkey]
        d_anon = results_default.district_label.loc[results_default.district == dkey].values[0]
        color = get_district_color(dkey)
        try:
            dres = district_res[scenarios[1]][dkey] - district_res[scenarios[0]][dkey]
            dres = dres.loc[dres.index.month == m].sum()
        except:
            try:
                dres = district_res[scenarios[1]][dkey]
                dres = dres.loc[dres.index.month == m].sum()
            except:
                try:
                    dres = -district_res[scenarios[0]][dkey]
                    dres = dres.loc[dres.index.month == m].sum()
                except:
                    pass
        # divide by 50 to get annual changes
        dres /= 50
        amt = 0.0
        for c in ['friant1', 'friant2', 'cvpdelta', 'exchange', 'cvc', 'kern', 'tule', 'kaweah', 'kings', 'tableA']:
            dc = d + '_' + c + '_'
            for mode in ['delivery', 'flood', 'flood_irrigation']:
                try:
                    amt += dres[dc + mode]
                except:
                    pass
        for cs, sign in zip([['exchanged_SW','exchanged_GW']], [[1,-1]]):
            for i,c in enumerate(cs):
                dc = d + '_' + c
                try:
                    amt += dres[dc] * sign[i]
                except:
                    pass
                
        if amt > 0.0:
            dlist.append('+' + d_anon)
            alist.append(amt)
            slist.append(1)
            mlist.append('+' + str(m))
            colorlist.append(color)

        elif amt < 0.0:
            dlist.append('-' + d_anon)
            alist.append(abs(amt))
            slist.append(-1)
            mlist.append('-' + str(m))
            colorlist.append(color)
                
df = pd.DataFrame({'Month': mlist, 'District': dlist, 'Magnitude': alist, 'Sign': slist, 'Color': colorlist})
print(df)
df.to_csv('figures/alluvial_month.csv', index=False)

    Month District  Magnitude  Sign    Color
0      +1    +O-10   0.001864     1  #A13403
1      +1    +O-26   0.003600     1  #E45709
2      -1     -O-6   0.012836    -1  #912E03
3      +1    +O-29   0.004850     1  #EC620F
4      -1    -O-24   0.002423    -1  #DF5106
..    ...      ...        ...   ...      ...
640   -12    -O-23   0.002281    -1  #DD4E04
641   +12    +O-17   0.000077     1  #C44001
642   +12     +O-7   0.020091     1  #962F03
643   +12     +O-8   0.003196     1  #993103
644   +12     +O-3   0.032872     1  #862903

[645 rows x 5 columns]


In [11]:
(df.Magnitude * df.Sign).sum()

36.08632272419392

In [12]:
### now total flows by season, b4 improvement
dlist = []
alist = []
slist = []
mlist = []
colorlist = []
season_dict = {'Oct-Dec':[10,11,12],'Jan-Mar':[1,2,3],'Apr-Jun':[4,5,6],'Jul-Sep':[7,8,9]}
for season, months in season_dict.items():
    for dkey in district_res[scenarios[1]]:
        d = district_lookup[dkey]
        d_anon = results_default.district_label.loc[results_default.district == dkey].values[0]
        color = get_district_color(dkey)
        try:
            dres = district_res[scenarios[0]][dkey]
            dres = dres.loc[[m in months for m in dres.index.month]].sum()
        except:
            pass
        # divide by 50 to get annual changes
        dres /= 50
        amt = 0.0
        for c in ['friant1', 'friant2', 'cvpdelta', 'exchange', 'cvc', 'kern', 'tule', 'kaweah', 'kings', 'tableA']:
            dc = d + '_' + c + '_'
            for mode in ['delivery', 'flood', 'flood_irrigation']:
                try:
                    amt += dres[dc + mode]
                except:
                    pass
        for cs, sign in zip([['exchanged_SW','exchanged_GW']], [[1,-1]]):
            for i,c in enumerate(cs):
                dc = d + '_' + c
                try:
                    amt += dres[dc] * sign[i]
                except:
                    pass
        if amt > 0.0:
            dlist.append('+' + d_anon)
            alist.append(amt)
            slist.append(1)
            mlist.append('+' + str(season))
            colorlist.append(color)

        elif amt < 0.0:
            dlist.append('-' + d_anon)
            alist.append(abs(amt))
            slist.append(-1)
            mlist.append('-' + str(season))
            colorlist.append(color)
                
df = pd.DataFrame({'Season': mlist, 'District': dlist, 'Magnitude': alist, 'Sign': slist, 'Color': colorlist})
print(df)
df.to_csv('figures/alluvial_season_b4.csv', index=False)

       Season District   Magnitude  Sign    Color
0    +Oct-Dec    +O-10    7.761468     1  #A13403
1    +Oct-Dec    +O-26    4.015708     1  #E45709
2    +Oct-Dec     +O-6   16.429952     1  #912E03
3    +Oct-Dec    +O-29    8.340349     1  #EC620F
4    +Oct-Dec    +O-24    6.161459     1  #DF5106
..        ...      ...         ...   ...      ...
211  +Jul-Sep    +O-23    0.084501     1  #DD4E04
212  +Jul-Sep    +O-17    4.091419     1  #C44001
213  +Jul-Sep     +O-7   63.722355     1  #962F03
214  +Jul-Sep     +O-8   41.060065     1  #993103
215  +Jul-Sep     +O-3  244.223320     1  #862903

[216 rows x 5 columns]


In [13]:
### now gain/loss by season
dlist = []
alist = []
slist = []
mlist = []
colorlist = []
season_dict = {'Oct-Dec':[10,11,12],'Jan-Mar':[1,2,3],'Apr-Jun':[4,5,6],'Jul-Sep':[7,8,9]}
for season, months in season_dict.items():
    for dkey in district_res[scenarios[1]]:
        d = district_lookup[dkey]
        d_anon = results_default.district_label.loc[results_default.district == dkey].values[0]
        color = get_district_color(dkey)
        try:
            dres = district_res[scenarios[1]][dkey] - district_res[scenarios[0]][dkey]
            dres = dres.loc[[m in months for m in dres.index.month]].sum()
        except:
            try:
                dres = district_res[scenarios[1]][dkey]
                dres = dres.loc[[m in months for m in dres.index.month]].sum()
            except:
                try:
                    dres = -district_res[scenarios[0]][dkey]
                    dres = dres.loc[[m in months for m in dres.index.month]].sum()
                except:
                    pass
        # divide by 50 to get annual changes
        dres /= 50
        amt = 0.0
        for c in ['friant1', 'friant2', 'cvpdelta', 'exchange', 'cvc', 'kern', 'tule', 'kaweah', 'kings', 'tableA']:
            dc = d + '_' + c + '_'
            for mode in ['delivery', 'flood', 'flood_irrigation']:
                try:
                    amt += dres[dc + mode]
                except:
                    pass
        for cs, sign in zip([['exchanged_SW','exchanged_GW']], [[1,-1]]):
            for i,c in enumerate(cs):
                dc = d + '_' + c
                try:
                    amt += dres[dc] * sign[i]
                except:
                    pass
        if amt > 0.0:
            dlist.append('+' + d_anon)
            alist.append(amt)
            slist.append(1)
            mlist.append('+' + str(season))
            colorlist.append(color)

        elif amt < 0.0:
            dlist.append('-' + d_anon)
            alist.append(abs(amt))
            slist.append(-1)
            mlist.append('-' + str(season))
            colorlist.append(color)
                
df = pd.DataFrame({'Season': mlist, 'District': dlist, 'Magnitude': alist, 'Sign': slist, 'Color': colorlist})
print(df)
df.to_csv('figures/alluvial_season.csv', index=False)

       Season District  Magnitude  Sign    Color
0    -Oct-Dec    -O-10   0.007595    -1  #A13403
1    -Oct-Dec    -O-26   0.016512    -1  #E45709
2    -Oct-Dec     -O-6   0.004988    -1  #912E03
3    +Oct-Dec    +O-29   0.005444     1  #EC620F
4    -Oct-Dec    -O-24   0.030630    -1  #DF5106
..        ...      ...        ...   ...      ...
211  -Jul-Sep    -O-23   0.011123    -1  #DD4E04
212  -Jul-Sep    -O-17   0.000746    -1  #C44001
213  +Jul-Sep     +O-7   0.034373     1  #962F03
214  +Jul-Sep     +O-8   0.023082     1  #993103
215  +Jul-Sep     +O-3   0.232284     1  #862903

[216 rows x 5 columns]


In [14]:
(df.Magnitude * df.Sign).sum()

36.08632272419392

In [15]:
### now total flows by wateryear, b4 improvement
dlist = []
alist = []
slist = []
wylist = []
colorlist = []
for wyi, wy in zip(range(1, 51), range(1902,1952)):
    for dkey in district_res[scenarios[1]]:
        d = district_lookup[dkey]
        d_anon = results_default.district_label.loc[results_default.district == dkey].values[0]
        color = get_district_color(dkey)
        try:
            dres = district_res[scenarios[0]][dkey]
            iswy = np.logical_or(np.logical_and(dres.index.month > 9, dres.index.year == wy-1), np.logical_and(dres.index.month <= 9, dres.index.year == wy))
            dres = dres.loc[iswy].sum()
        except:
            pass
        # divide by 50 to get annual changes
        dres /= 50
        amt = 0.0
        for c in ['friant1', 'friant2', 'cvpdelta', 'exchange', 'cvc', 'kern', 'tule', 'kaweah', 'kings', 'tableA']:
            dc = d + '_' + c + '_'
            for mode in ['delivery', 'flood', 'flood_irrigation']:
                try:
                    amt += dres[dc + mode]
                except:
                    pass
        for cs, sign in zip([['exchanged_SW','exchanged_GW']], [[1,-1]]):
            for i,c in enumerate(cs):
                dc = d + '_' + c
                try:
                    amt += dres[dc] * sign[i]
                except:
                    pass
        if amt > 0.0:
            dlist.append('+' + d_anon)
            alist.append(amt)
            slist.append(1)
            wylist.append('+' + str(wyi))
            colorlist.append(color)

        elif amt < 0.0:
            dlist.append('-' + d_anon)
            alist.append(abs(amt))
            slist.append(-1)
            wylist.append('-' + str(wyi))
            colorlist.append(color)
                
df = pd.DataFrame({'Wateryear': wylist, 'District': dlist, 'Magnitude': alist, 'Sign': slist, 'Color': colorlist})
print(df)
df.to_csv('figures/alluvial_wateryear_b4.csv', index=False)

     Wateryear District  Magnitude  Sign    Color
0           +1    +O-10   0.841386     1  #A13403
1           +1    +O-26   0.506588     1  #E45709
2           +1     +O-6   3.962778     1  #912E03
3           +1    +O-29   1.769687     1  #EC620F
4           +1    +O-24   0.908499     1  #DF5106
...        ...      ...        ...   ...      ...
2657       +50    +O-28   0.948264     1  #E95F0D
2658       +50    +O-17   0.283601     1  #C44001
2659       +50     +O-7   4.049815     1  #962F03
2660       +50     +O-8   2.711862     1  #993103
2661       +50     +O-3   5.105597     1  #862903

[2662 rows x 5 columns]


In [16]:
### now gain/loss by wateryear
dlist = []
alist = []
slist = []
wylist = []
colorlist = []
for wyi, wy in zip(range(1, 51), range(1902,1952)):
    for dkey in district_res[scenarios[1]]:
        d = district_lookup[dkey]
        d_anon = results_default.district_label.loc[results_default.district == dkey].values[0]
        color = get_district_color(dkey)
        try:
            dres = district_res[scenarios[1]][dkey] - district_res[scenarios[0]][dkey]
            iswy = np.logical_or(np.logical_and(dres.index.month > 9, dres.index.year == wy-1), np.logical_and(dres.index.month <= 9, dres.index.year == wy))
            dres = dres.loc[iswy].sum()
        except:
            try:
                dres = district_res[scenarios[1]][dkey]
                iswy = np.logical_or(np.logical_and(dres.index.month > 9, dres.index.year == wy-1), np.logical_and(dres.index.month <= 9, dres.index.year == wy))
                dres = dres.loc[iswy].sum()
            except:
                try:
                    dres = -district_res[scenarios[0]][dkey]
                    iswy = np.logical_or(np.logical_and(dres.index.month > 9, dres.index.year == wy-1), np.logical_and(dres.index.month <= 9, dres.index.year == wy))
                    dres = dres.loc[iswy].sum()
                except:
                    pass
        # divide by 50 to get annual changes
        dres /= 50
        amt = 0.0
        for c in ['friant1', 'friant2', 'cvpdelta', 'exchange', 'cvc', 'kern', 'tule', 'kaweah', 'kings', 'tableA']:
            dc = d + '_' + c + '_'
            for mode in ['delivery', 'flood', 'flood_irrigation']:
                try:
                    amt += dres[dc + mode]
                except:
                    pass
        for cs, sign in zip([['exchanged_SW','exchanged_GW']], [[1,-1]]):
            for i,c in enumerate(cs):
                dc = d + '_' + c
                try:
                    amt += dres[dc] * sign[i]
                except:
                    pass
        if amt > 0.0:
            dlist.append('+' + d_anon)
            alist.append(amt)
            slist.append(1)
            wylist.append('+' + str(wyi))
            colorlist.append(color)

        elif amt < 0.0:
            dlist.append('-' + d_anon)
            alist.append(abs(amt))
            slist.append(-1)
            wylist.append('-' + str(wyi))
            colorlist.append(color)
                
df = pd.DataFrame({'Wateryear': wylist, 'District': dlist, 'Magnitude': alist, 'Sign': slist, 'Color': colorlist})
print(df)
df.to_csv('figures/alluvial_wateryear.csv', index=False)

     Wateryear District     Magnitude  Sign    Color
0           -1    -O-10  2.359167e-04    -1  #A13403
1           +1    +O-26  3.602565e-06     1  #E45709
2           +1     +O-6  2.677405e-03     1  #912E03
3           -1    -O-29  2.255173e-03    -1  #EC620F
4           +1    +O-24  4.286610e-05     1  #DF5106
...        ...      ...           ...   ...      ...
2613       +50     +F-5  2.170132e-02     1  #08509A
2614       -50    -O-21  1.651472e-04    -1  #D74701
2615       -50    -O-28  2.633199e-03    -1  #E95F0D
2616       -50    -O-17  7.518110e-07    -1  #C44001
2617       -50     -O-3  3.979885e-02    -1  #862903

[2618 rows x 5 columns]


In [17]:
(df.Magnitude * df.Sign).sum()

36.086322724193906

In [18]:
### get rank order of reservoir inflows for millerton
dat = get_results_sensitivity_number_outside_model('FKC_experiment/FKC_experiment_capow_50yr_' + scenarios[0] + '/results.hdf5', '')
# keys = list(dat.keys())
# index = dat.index
# year = index.year
# month = index.month
# dom = index.day
# doy = index.dayofyear
# dowy = (doy + (365-274)) % 365
# wy = np.array([year[i] if month[i] < 10 else year[i] + 1 for i in range(len(year))])
#     for d, v in district_lookup.items():
#         b = [k for k in keys if (((d in k.split('_')) or (v in k.split('_'))) and (('delivery' in k.split('_')) or ('flood' in k.split('_')) or ('recharged' in k.split('_')) or ('exchanged' in k.split('_')) or ('inleiu' in k.split('_')) or ('peiupumping' in k.split('_')) or ('banked' in k.split('_'))))]
#         df = dat.loc[:, b]
#         ## undo summation over years
#         for y in range(df.index.year.min() + 2, df.index.year.max() + 1):
#             maxprevious = df.loc[wy < y, :].iloc[-1, :]
#             df.loc[wy == y, :] += maxprevious
#         district_res[s][d] = df.diff()
#     del dat, df

dat = dat.millerton_Q
index = dat.index
year = index.year
month = index.month
dom = index.day
doy = index.dayofyear
dowy = (doy + (365-274)) % 365
wy = np.array([year[i] if month[i] < 10 else year[i] + 1 for i in range(len(year))])

datAgg = dat.groupby(wy).sum()

p75 = np.percentile(datAgg, 75)
p50 = np.percentile(datAgg, 50)
p25 = np.percentile(datAgg, 25)

wetness_dict = {y: 'q4' if x >= p75 else 'q3' if x >= p50 else 'q2'  if x >= p25 else 'q1' for x,y in zip(datAgg, datAgg.index)}
wetness_list = [wetness_dict[y] for y in wy]

In [19]:
### now total flows by wateryear wetness, b4 improvment
dlist = []
alist = []
slist = []
wlist = []
colorlist = []
for wetness in ['q1','q2','q3','q4']:
    for dkey in district_res[scenarios[1]]:
        d = district_lookup[dkey]
        d_anon = results_default.district_label.loc[results_default.district == dkey].values[0]
        color = get_district_color(dkey)
        try:
            dres = district_res[scenarios[0]][dkey]
            dres = dres.loc[[w == wetness for w in wetness_list]].sum()
        except:
            pass
        # divide by 50 to get annual changes
        dres /= 50
        amt = 0.0
        for c in ['friant1', 'friant2', 'cvpdelta', 'exchange', 'cvc', 'kern', 'tule', 'kaweah', 'kings', 'tableA']:
            dc = d + '_' + c + '_'
            for mode in ['delivery', 'flood', 'flood_irrigation']:
                try:
                    amt += dres[dc + mode]
                except:
                    pass
        for cs, sign in zip([['exchanged_SW','exchanged_GW']], [[1,-1]]):
            for i,c in enumerate(cs):
                dc = d + '_' + c
                try:
                    amt += dres[dc] * sign[i]
                except:
                    pass
        if amt > 0.0:
            dlist.append('+' + d_anon)
            alist.append(amt)
            slist.append(1)
            wlist.append('+' + str(wetness))
            colorlist.append(color)

        elif amt < 0.0:
            dlist.append('-' + d_anon)
            alist.append(abs(amt))
            slist.append(-1)
            wlist.append('-' + str(wetness))
            colorlist.append(color)
                
df = pd.DataFrame({'HydroQuartile': wlist, 'District': dlist, 'Magnitude': alist, 'Sign': slist, 'Color': colorlist})
print(df)
df.to_csv('figures/alluvial_wetness_b4.csv', index=False)

    HydroQuartile District   Magnitude  Sign    Color
0             +q1    +O-10    5.773392     1  #A13403
1             +q1    +O-26    4.283536     1  #E45709
2             +q1     +O-6   26.642854     1  #912E03
3             +q1    +O-29    9.308659     1  #EC620F
4             +q1    +O-24   10.430499     1  #DF5106
..            ...      ...         ...   ...      ...
211           +q4    +O-23    0.733177     1  #DD4E04
212           +q4    +O-17    3.879198     1  #C44001
213           +q4     +O-7   75.578111     1  #962F03
214           +q4     +O-8   34.424705     1  #993103
215           +q4     +O-3  421.354670     1  #862903

[216 rows x 5 columns]


In [20]:
### now gain/loss by wateryear wetness
dlist = []
alist = []
slist = []
wlist = []
colorlist = []
for wetness in ['q1','q2','q3','q4']:
    for dkey in district_res[scenarios[1]]:
        d = district_lookup[dkey]
        d_anon = results_default.district_label.loc[results_default.district == dkey].values[0]
        color = get_district_color(dkey)
        try:
            dres = district_res[scenarios[1]][dkey] - district_res[scenarios[0]][dkey]
            dres = dres.loc[[w == wetness for w in wetness_list]].sum()
        except:
            try:
                dres = district_res[scenarios[1]][dkey]
                dres = dres.loc[[w == wetness for w in wetness_list]].sum()
            except:
                try:
                    dres = -district_res[scenarios[0]][dkey]
                    dres = dres.loc[[w == wetness for w in wetness_list]].sum()
                except:
                    pass
        # divide by 50 to get annual changes
        dres /= 50
        amt = 0.0
        for c in ['friant1', 'friant2', 'cvpdelta', 'exchange', 'cvc', 'kern', 'tule', 'kaweah', 'kings', 'tableA']:
            dc = d + '_' + c + '_'
            for mode in ['delivery', 'flood', 'flood_irrigation']:
                try:
                    amt += dres[dc + mode]
                except:
                    pass
        for cs, sign in zip([['exchanged_SW','exchanged_GW']], [[1,-1]]):
            for i,c in enumerate(cs):
                dc = d + '_' + c
                try:
                    amt += dres[dc] * sign[i]
                except:
                    pass
        if amt > 0.0:
            dlist.append('+' + d_anon)
            alist.append(amt)
            slist.append(1)
            wlist.append('+' + str(wetness))
            colorlist.append(color)

        elif amt < 0.0:
            dlist.append('-' + d_anon)
            alist.append(abs(amt))
            slist.append(-1)
            wlist.append('-' + str(wetness))
            colorlist.append(color)

df = pd.DataFrame({'HydroQuartile': wlist, 'District': dlist, 'Magnitude': alist, 'Sign': slist, 'Color': colorlist})
print(df)
df.to_csv('figures/alluvial_wetness.csv', index=False)

    HydroQuartile District  Magnitude  Sign    Color
0             -q1    -O-10   0.002121    -1  #A13403
1             -q1    -O-26   0.020581    -1  #E45709
2             +q1     +O-6   0.050421     1  #912E03
3             -q1    -O-29   0.008637    -1  #EC620F
4             -q1    -O-24   0.000212    -1  #DF5106
..            ...      ...        ...   ...      ...
211           -q4    -O-23   0.005519    -1  #DD4E04
212           -q4    -O-17   0.000250    -1  #C44001
213           +q4     +O-7   0.000096     1  #962F03
214           +q4     +O-8   0.002042     1  #993103
215           -q4     -O-3   0.055628    -1  #862903

[216 rows x 5 columns]


In [21]:
(df.Magnitude * df.Sign).sum()

36.086322724193906

### trying to match gains/losses from get_objectives.py

In [22]:
### read in data & organize datetimes
def get_district_data(datafile):
  dat = get_results_sensitivity_number_outside_model(datafile, '')
  keys = list(dat.keys())
  index = dat.index
  year = index.year
  month = index.month
  # dom = index.day
  # doy = index.dayofyear
  # dowy = (doy + (365-274)) % 365
  wy = np.array([year[i] if month[i] < 10 else year[i] + 1 for i in range(len(year))])
  ### get relevant results for each district
  districts = {}
  for d, v in district_lookup.items():
    b = [k for k in keys if (((d in k.split('_')) or (v in k.split('_'))) and (('delivery' in k.split('_')) or ('flood' in k.split('_')) or ('recharged' in k.split('_')) or \
                                                                                ('exchanged' in k.split('_')) or ('inleiu' in k.split('_')) or ('leiupumping' in k.split('_')) or \
                                                                                  ('banked' in k.split('_'))))]
    df = dat.loc[:, b]
    ## undo summation over years
    for y in range(df.index.year.min() + 2, df.index.year.max() + 1):
      maxprevious = df.loc[wy < y, :].iloc[-1, :]
      df.loc[wy == y, :] += maxprevious
    df.iloc[1:, :] = df.diff().iloc[1:, :]
    ## get total new surface water deliveries = *district*_*contract*_delivery + *district*_*contract*_flood + *district*_*contract*_flood_irrigation - *district*_exchanged_GW + *district*_exchanged_SW
    df['new_deliveries'] = 0.0
    for (wtype, position) in [('delivery', 2), ('flood', 2), ('SW', 2)]:
      for c in df.columns:
        try:
          if c.split('_')[position] == wtype:
            df['new_deliveries'] += df[c]
        except:
          pass
    for (wtype, position) in [('GW', 2)]:
      for c in df.columns:
        try:
          if c.split('_')[position] == wtype:
            df['new_deliveries'] -= df[c]
        except:
          pass
    ## get irrigation deliveries for drought year metric = *district*_*contract*_delivery - *district*_*contract*_recharged + *district*_*contract*_flood_irrigation + *district*_recover_banked + *district*_exchanged_SW + *district*_inleiu_irrigation
    df['irrig_deliveries'] = 0.0
    for (wtype, position) in [('delivery', 2), ('irrigation', 3), ('recover', 1), ('SW', 2), ('irrigation', 2)]:
      for c in df.columns:
        try:
          if c.split('_')[position] == wtype:
            df['irrig_deliveries'] += df[c]
        except:
          pass
    for (wtype, position) in [('recharged', 2)]:
      for c in df.columns:
        try:
          if c.split('_')[position] == wtype:
            df['irrig_deliveries'] -= df[c]
        except:
          pass

    districts[d] = df

  return districts, wy


In [23]:
s_districts = {}
for s in scenarios:
    s_districts[s], wy = get_district_data('FKC_experiment/FKC_experiment_capow_50yr_' + s + '/results.hdf5')


In [24]:
s = scenarios[1]
amt = 0
for d in s_districts[scenarios[1]]:
#     try:
#         if shares[s][d] > 0.0:
#             amt += (s_districts[s][d]['new_deliveries'] - s_districts[scenarios[0]][d]['new_deliveries']).sum()
#     except:
#         pass
    
    try:
        amt += (s_districts[s][d]['new_deliveries'] - s_districts[scenarios[0]][d]['new_deliveries']).sum()
    except:
        pass
amt/50

36.10010655499218

In [25]:
len(s_districts[scenarios[0]])

55